# Combinar conjuntos de datos: Concat y Append

Algunos de los estudios de datos más interesantes proceden de la combinación de distintas fuentes de datos.
Estas operaciones pueden implicar cualquier cosa, desde una concatenación muy directa de dos conjuntos de datos diferentes, hasta uniones y fusiones más complicadas al estilo de las bases de datos que manejan correctamente cualquier solapamiento entre los conjuntos de datos.
``Series`` y ``DataFrame`` se construyen con este tipo de operaciones en mente, y Pandas incluye funciones y métodos que hacen que este tipo de manipulación de datos sea rápida y sencilla.

Aquí echaremos un vistazo a la simple concatenación de ``Series`` y ``DataFrame``s con la función ``pd.concat``; más tarde nos sumergiremos en fusiones y uniones en memoria más sofisticadas implementadas en Pandas.

Comenzamos con las importaciones estándar:

In [ ]:
import pandas as pd
import numpy as np

Por conveniencia, definiremos esta función que crea un ``DataFrame`` de una forma particular que será útil más adelante:

In [ ]:
def make_df(cols, ind):
    """Crea rápidamente un DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# ejemplo DataFrame
make_df('ABC', range(3))

Además, crearemos una clase rápida que nos permita mostrar múltiples ``DataFrame`` uno al lado del otro. El código hace uso del método especial ``_repr_html_``, que IPython utiliza para implementar su visualización de objetos enriquecidos:

In [ ]:
class display(object):
    """Mostrar la representación HTML de varios objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

Su utilidad quedará más clara a medida que avancemos en el siguiente apartado.

## Recall: Concatenación de matrices NumPy

La concatenación de objetos ``Series`` y ``DataFrame`` es muy similar a la concatenación de arrays Numpy, que se puede hacer a través de la función ``np.concatenate`` como se discutió en [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb).
Recuerda que con ella puedes combinar el contenido de dos o más arrays en un único array:

In [ ]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

El primer argumento es una lista o tupla de matrices a concatenar.
Además, toma una palabra clave ``axis`` que permite especificar el eje a lo largo del cual se concatenará el resultado:

In [ ]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

## Concatenación simple con ``pd.concat``

``pd.concat()`` can be used for a simple concatenation of ``Series`` or ``DataFrame`` objects, just as ``np.concatenate()`` can be used for simple concatenations of arrays:
Pandas tiene una función, ``pd.concat()``, que tiene una sintaxis similar a ``np.concatenate`` pero contiene una serie de opciones que discutiremos momentáneamente:

```python
# Firma en Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=Verdadero)
```

``pd.concat()`` puede utilizarse para una simple concatenación de objetos ``Series`` o ``DataFrame``, al igual que ``np.concatenate()`` puede utilizarse para simples concatenaciones de arrays:

In [ ]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
print(ser1)
ser2 = pd.Series(['D', 'E', 'F'], index=[1, 2, 3])
print(ser2)
pd.concat([ser1, ser2], axis = 0).reset_index(drop=True)

También funciona para concatenar objetos de mayor dimensión, como ``DataFrame``s:

In [ ]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [1, 2])

display('df1','df2','pd.concat([df1, df2], axis=1)')

Por defecto, la concatenación se realiza por filas dentro del ``DataFrame`` (es decir, ``axis=0``).
Al igual que ``np.concatenate``, ``pd.concat`` permite especificar un eje a lo largo del cual se producirá la concatenación.

Considere el siguiente ejemplo:

In [ ]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis=0)")

Podríamos haber especificado ``axis=1``; aquí hemos utilizado el más intuitivo ``axis='col'``.

### Duplicate indices

Una diferencia importante entre ``np.concatenate`` y ``pd.concat`` es que la concatenación en Pandas *preserva los índices*, ¡incluso si el resultado tiene índices duplicados!
Considera este sencillo ejemplo:

In [ ]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # ¡haz índices duplicados!
display('x', 'y', 'pd.concat([x, y], axis = 0)')

Observe los índices repetidos en el resultado.
Aunque esto es válido dentro de ``DataFrame``s, el resultado es a menudo indeseable.
``pd.concat()`` nos da algunas maneras de manejarlo.

#### Captura de las repeticiones como error

Si quieres simplemente verificar que los índices en el resultado de ``pd.concat()`` no se solapan, puedes especificar la bandera ``verify_integrity``.
Con True, la concatenación lanzará una excepción si hay índices duplicados.
He aquí un ejemplo, en el que para mayor claridad capturaremos e imprimiremos el mensaje de error:

In [ ]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

#### Ignorar el índice

A veces, el índice en sí no importa y se prefiere ignorarlo.
Esta opción puede especificarse utilizando el indicador ``ignorar_índice``.
Si se establece en true, la concatenación creará un nuevo índice entero para la ``Serie`` resultante:

In [ ]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

#### Añadir claves MultiIndex

Otra opción es utilizar la opción ``keys`` para especificar una etiqueta para las fuentes de datos; el resultado será una serie indexada jerárquicamente que contendrá los datos:

In [ ]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

El resultado es un ``DataFrame`` con múltiples índices, y podemos utilizar las herramientas comentadas en [Hierarchical Indexing](05_Hierarchical-Indexing.ipynb) para transformar estos datos en la representación que nos interesa.

### Concatenación con uniones

En los ejemplos sencillos que acabamos de ver, concatenábamos principalmente ``DataFrame``s con nombres de columna compartidos.
En la práctica, los datos de diferentes fuentes pueden tener diferentes conjuntos de nombres de columna, y ``pd.concat`` ofrece varias opciones en este caso.
Considera la concatenación de los siguientes dos ``DataFrame``s, que tienen algunas (¡pero no todas!) columnas en común:

In [ ]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
# display(df5)
# display(df6)
display('df5', 'df6', 'pd.concat([df5, df6])')

Por defecto, las entradas para las que no hay datos disponibles se rellenan con valores NA.
Para cambiar esto, podemos especificar una de varias opciones para los parámetros ``join`` y ``join_axes`` de la función concatenar.
Por defecto, la unión es una unión de las columnas de entrada (``join='outer'``), pero podemos cambiar esto a una intersección de las columnas usando ``join='inner'``:

In [ ]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

La combinación de opciones de la función ``pd.concat`` permite una amplia gama de comportamientos posibles al unir dos conjuntos de datos; téngalos en cuenta cuando utilice estas herramientas para sus propios datos.